<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Ada Tech [DS-PY-004] Técnicas de Programação I (PY) Aulas 4 e 5 : Pandas - Resolução do Exercício 3.

## Intro

Nesta parte vimos alguns métodos do Pandas para calcular medidas por grupos.

Para os exercícios, usaremos um conjunto de dados de propriedades à venda na Cidade de Buenos Aires em 2016 disponibilizado pelo GCBA.

A ideia dessa prática é revisar alguns tópicos que vimos nas aulas anteriores e exercitar os diferentes métodos que vimos nesta prática e que vamos usar novamente em aulas futuras.

## Dataset

O Governo da Cidade de Buenos Aires disponibiliza alguns [conjuntos de dados](https://data.buenosaires.gob.ar/) para acesso público

Lá encontramos dados sobre [imóveis à venda](https://data.buenosaires.gob.ar/dataset/departamentos-venta), separados por ano.

Nesta aula, vamos usar um subconjunto dos dados de [Apartamentos à venda 2016](https://data.buenosaires.gob.ar/dataset/departamentos-venta/archivo/juqdkmgo-7031-resource) que baixamos.

Na pasta Dados desta aulas, já temos esse conjunto de dados baixado, o nome do arquivo é **departamentos-em-venda-2016.csv**.

## Imports

In [1]:
import pandas as pd
import numpy as np

## Exercício 1  - Importar 

Vamos ler os dados do arquivo `departamentos-em-venda-2016.csv`.

Vamos ver quantos registros você possui e quais são os tipos de dados em cada coluna.

Vejamos os primeiros registros para verificar se os dados foram importados corretamente.

In [2]:
data_location = "../Data/departamentos-em-venda-2016.csv"
data = pd.read_csv(data_location, sep = ";")
print(data.shape)
print(data.dtypes)
data.head(3)

(7564, 29)
CALLE                       object
NUMERO                     float64
ID_ZONAPRO                   int64
OPERACION                   object
TIPO                        object
M2                           int64
M2CUB                        int64
PRECIOTEXT                  object
PRECIOARS                    int64
PRECIOARSM                   int64
DOLARES                      int64
U_S_M2                       int64
AMBIENTES                    int64
ANTIGUEDAD                   int64
BAÑOS                        int64
DIRECCION                   object
LOCATION                    object
PUBLICADO                   object
PROCESADO                   object
URL                         object
REVISION                   float64
NOTA                       float64
DIRECCION_NORMALIZADA       object
BARRIO                      object
COMUNA                      object
CODIGO_POSTAL              float64
CODIGO_POSTAL_ARGENTINO     object
LATITUD                    float64
LONGITUD 

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,URL,REVISION,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD
0,GUATEMALA,5574.0,42408691,VTA,DTO,57,50,U$S 170.150,2977625,59553,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42408...,NaN,NaN,GUATEMALA 5574,PALERMO,COMUNA 14,1425.0,C1425BVH,-34.580581,-58.431758
1,ZAPATA,300.0,42408710,VTA,DTO,46,46,U$S 118.650,2076375,45139,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42408...,NaN,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609
2,ZAPATA,300.0,42518390,VTA,DTO,61,56,U$S 181.470,3175725,56709,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42518...,NaN,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609


## Exercício 2  - Limpeza

Vamos criar uma nova coluna (PRECIOTEXT_CLEAN) do tipo numérico, que tem os valores em dólares que lemos no campo PRECIOTEXT, ou NaN se esses valores forem expressos em pesos.

1) Crie um objeto Series que resulte da limpeza dos valores do campo PRECIOTEXT. Temos que remover os símbolos "U\\$S" e "."

2) No caso da Series resultante do ponto 1) substituímos os valores em pesos ('\\$') por nulos.

3) Crie uma nova coluna de tipo numérico no DataFrame (PRECIOTEXT_CLEAN) e atribua os valores de resultado de 2)

In [3]:
import re

# substituição de U$S
dolares_pattern = "U\$S\s*"
dolares_regex = re.compile(dolares_pattern)

# substituo o ponto decimal
decimal_pattern = "\."
decimal_regex = re.compile(decimal_pattern)

without_dolar = data.PRECIOTEXT.apply(lambda x: x if x is np.NaN else dolares_regex.sub("", x))

without_dolar_without_dot = without_dolar.apply(lambda x: x if x is np.NaN else decimal_regex.sub("", x))

type(without_dolar_without_dot)

pandas.core.series.Series

In [4]:
# monto uma máscara com os registros que têm o valor de PRECIOTEXT em $
pesos_pattern = "\$"
pesos_regex = re.compile(pesos_pattern)

matches_pesos = without_dolar_without_dot.apply(lambda x: x if x is np.NaN else pesos_regex.match(x))
pesos_mask = matches_pesos.notnull()


print(without_dolar_without_dot[pesos_mask])


without_dolar_without_dot[pesos_mask] = np.NaN


142        $13000
156     $ 1140000
257        $89000
278     $ 1396000
279     $ 1927000
          ...    
6556    $ 1021734
6557    $ 1188007
6558    $ 1093950
6559    $ 1077120
6560    $ 1108868
Name: PRECIOTEXT, Length: 308, dtype: object


In [5]:
without_dolar_without_dot_without_pesos_num  = without_dolar_without_dot.astype(float)
data["PRECIOTEXT_CLEAN"] = without_dolar_without_dot_without_pesos_num


## Exercício 3  - Preço por metro quadrado em dólares

Vamos criar uma coluna do tipo numérico (PRECIOTEXTM) que tem o preço do $m^{2}$ em dólares calculado como o valor de PRECIOTEXT_CLEAN / M2.

In [6]:
data["PRECIOTEXTM"] = (data["PRECIOTEXT_CLEAN"] / data["M2"]).round(2)
data.head(3)


,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD,PRECIOTEXT_CLEAN,PRECIOTEXTM
0,GUATEMALA,5574.0,42408691,VTA,DTO,57,50,U$S 170.150,2977625,59553,...,NaN,GUATEMALA 5574,PALERMO,COMUNA 14,1425.0,C1425BVH,-34.580581,-58.431758,170150.0,2985.09
1,ZAPATA,300.0,42408710,VTA,DTO,46,46,U$S 118.650,2076375,45139,...,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609,118650.0,2579.35
2,ZAPATA,300.0,42518390,VTA,DTO,61,56,U$S 181.470,3175725,56709,...,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609,181470.0,2974.92


## Exercício 4  - Preço médio por metro quadrado em dólares por bairro

Vamos calcular o preço médio por metro quadrado em dólares por bairro usando `groupby` e `pivot_table`.

Quais são os cinco bairros mais caros?

In [7]:
preco_m2_por_bairro = data.groupby("BARRIO")["PRECIOTEXTM"].mean().round(2)
#print(precio_m2_prom_barrio)
#type(precio_m2_prom_barrio)
preco_m2_por_bairro_sort = preco_m2_por_bairro.sort_values(ascending = False)
preco_m2_por_bairro_top5 = preco_m2_por_bairro_sort[0:5]
preco_m2_por_bairro_top5

BARRIO
PUERTO MADERO    5412.47
PALERMO          3136.02
BELGRANO         3110.07
RETIRO           2941.05
RECOLETA         2876.20
Name: PRECIOTEXTM, dtype: float64

In [8]:
preco_m2_por_bairro = data.pivot_table(index = 'BARRIO', aggfunc = {'PRECIOTEXTM':'mean'}) 
preco_m2_por_bairro_sort = preco_m2_por_bairro.sort_values(by = "PRECIOTEXTM", ascending = False)
preco_m2_por_bairro_top5 = preco_m2_por_bairro_sort[0:5]
preco_m2_por_bairro_top5

,PRECIOTEXTM
BARRIO,
PUERTO MADERO,5412.466964
PALERMO,3136.015176
BELGRANO,3110.066058
RETIRO,2941.053226
RECOLETA,2876.201798


## Exercício 5  - agrupar por campos numéricos (decis)

Vamos construir os decis da superfície de dados usando o método `qcut` do numpy.

Isso associa uma categoria a cada registro de dados, onde o valor dessa categoria indica a qual decil esse registro pertence.

Usando o método `groupby`, vamos calcular o preço médio por $m^{2}$ em dólares para cada um dos decis de superfície.

In [9]:
q_superficie = pd.qcut(data.M2, 10)
q_superficie

0         (54.0, 64.0]
1         (43.0, 48.0]
2         (54.0, 64.0]
3       (123.0, 730.0]
4         (35.0, 39.0]
             ...      
7559      (35.0, 39.0]
7560      (43.0, 48.0]
7561      (43.0, 48.0]
7562      (43.0, 48.0]
7563      (48.0, 54.0]
Name: M2, Length: 7564, dtype: category
Categories (10, interval[float64, right]): [(14.999, 35.0] < (35.0, 39.0] < (39.0, 43.0] < (43.0, 48.0] ... (64.0, 74.0] < (74.0, 88.0] < (88.0, 123.0] < (123.0, 730.0]]

In [10]:
data.groupby(q_superficie)["PRECIOTEXTM"].median()

M2
(14.999, 35.0]    2452.660
(35.0, 39.0]      2283.485
(39.0, 43.0]      2300.000
(43.0, 48.0]      2377.780
(48.0, 54.0]      2283.020
(54.0, 64.0]      2374.965
(64.0, 74.0]      2302.175
(74.0, 88.0]      2375.000
(88.0, 123.0]     2552.685
(123.0, 730.0]    2692.310
Name: PRECIOTEXTM, dtype: float64

## Exercício 6  - agrupar por campos do tipo string

Usando o método `groupby`, vamos calcular o preço médio por $m^{2}$ em dólares para cada um dos bairros da cidade.

In [11]:
data.groupby("BARRIO")["PRECIOTEXTM"].median()

BARRIO
AGRONOMIA            2375.000
ALMAGRO              2226.190
BALVANERA            1977.270
BARRACAS             2241.875
BELGRANO             2983.775
BOCA                 1764.635
BOEDO                2340.430
CABALLITO            2419.145
CHACARITA            2383.720
COGHLAN              2543.210
COLEGIALES           2506.985
CONSTITUCION         2262.230
FLORES               1842.110
FLORESTA             1694.445
LINIERS              1921.570
MATADEROS            1666.670
MONSERRAT            2328.375
MONTE CASTRO         2055.880
NUEVA POMPEYA        1622.725
NUÑEZ                2417.425
PALERMO              2906.250
PARQUE AVELLANEDA    1812.500
PARQUE CHACABUCO     2656.360
PARQUE CHAS          2045.450
PARQUE PATRICIOS     1977.160
PATERNAL             1720.780
PUERTO MADERO        5074.350
RECOLETA             2752.810
RETIRO               2922.080
SAAVEDRA             2287.590
SAN CRISTOBAL        1836.360
SAN NICOLAS          2142.860
SAN TELMO            2035.505
VEL

## Exercício 7 - agrupamento por condições personalizadas

Vamos definir uma máscara booleana que é verdadeira para os registros cujo preço em dólares é de até 150.000

Vamos agrupar por este critério e calcular o valor da área mediana para esses dois grupos (preço $\le 150.000$ e preço $ \lt 150.000$).

In [12]:
mask_price = data.PRECIOTEXT_CLEAN <= 150000

data.groupby(mask_price)["M2"].median()

PRECIOTEXT_CLEAN
False    83.0
True     44.0
Name: M2, dtype: float64